In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [0]:
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.layers import Input, Flatten, Dense
from tensorflow.python.keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False) # data_format = "channels_first"

# Freeze the layers except the last 4 layers
for layer in model_vgg16_conv.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in model_vgg16_conv.layers:
    print(layer, layer.trainable)

#Create your own input format (here 3x200x200)
input = Input(shape=(150,150,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = Dense(1, activation='linear', name='pred')(x)

#Create your own model 
my_model = Model(input, x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training

my_model.compile(loss='mse',
              optimizer='adam',
              metrics=['mse'])
my_model.summary()

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f0c917ad710> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0c917b1d68> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0c90f57748> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0c90f197b8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0c90f19630> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0c90f2a6a0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0c90f2fb38> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0c90f2f9b0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0c90f39cf8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0c804ba438> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0c804c2358> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [0]:
# load data into dataframe with 'path' and 'target' columns
import pandas as pd
train_label_df = pd.read_csv('/content/drive/My Drive/team7/data.csv', delimiter=' ', header=1, names=["uid"])
train_label_df["path"] = train_label_df['uid'].apply(lambda x : '/content/drive/My Drive/team7/data/' + x.split(',')[-1][1:].split('/')[-1])
train_label_df["target"] = train_label_df['uid'].apply(lambda x : int(x.split(',')[1])/int(x.split(',')[2]))
train_label_df =train_label_df.drop(['uid'], axis=1)

train_label_df2 = pd.read_csv('/content/drive/My Drive/team7/data_temp.csv', delimiter=' ', header=1, names=["uid"])
train_label_df2["path"] = train_label_df2['uid'].apply(lambda x : "/content/drive/My Drive/team7/data/" + x.split(',')[0] + ".jpg")
train_label_df2["target"] = train_label_df2['uid'].apply(lambda x : int(x.split(',')[1])/int(x.split(',')[3]))
train_label_df2 =train_label_df2.drop(['uid'], axis=1)
#uid,likes,comments,followers
#B-A-bW9l1Gm,601,43,6715

In [0]:
train_df = pd.concat([train_label_df, train_label_df2], axis=0).reset_index().drop(['index'], axis=1)

In [0]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('/content/drive/My Drive/team7/data/') if isfile(join('/content/drive/My Drive/team7/data/', f))]

In [0]:
new_df = train_df
j = 0
for i in train_df['path']:
    #print(i.split("/")[-1])
    if i.split("/")[-1] not in onlyfiles:
        new_df = new_df.drop(train_df.index[train_df['path'] == i].tolist())
    else:
      j+=1
train_df = new_df.reset_index().drop(['index'], axis=1)
# All are found
train_df

,path,target
0,/content/drive/My Drive/team7/data/B9dZzI1n-EV...,0.112025
1,/content/drive/My Drive/team7/data/B9cQMfVnkMe...,0.638448
2,/content/drive/My Drive/team7/data/B9a0IQWnY9M...,0.053981
3,/content/drive/My Drive/team7/data/B9NnTUrHoNX...,0.143885
4,/content/drive/My Drive/team7/data/B9XuP83gx_j...,0.030143
...,...,...
2030,/content/drive/My Drive/team7/data/B_UgmnQj0tQ...,0.041773
2031,/content/drive/My Drive/team7/data/B_QoOOxl-ve...,0.024106
2032,/content/drive/My Drive/team7/data/B_OM7wVoCUd...,0.025871
2033,/content/drive/My Drive/team7/data/B_QTlktFkRZ...,0.014322


In [0]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(train_df, test_size=0.2)

In [0]:
batch_size = 128
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator()

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator()

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,
        x_col = 'path',
        y_col = 'target',
        # directory='data',
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='other',
       #  preprocessing_function=lambda x: x print(x)
)  # since we use binary_crossentropy loss, we need binary labels

test_generator = test_datagen.flow_from_dataframe(
        dataframe=test,
        x_col = 'path',
        y_col = 'target',
        # directory='data',
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='other',
)

Found 1628 validated image filenames.
Found 407 validated image filenames.


In [0]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())  

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']


In [0]:
my_model.save_weights('/content/drive/My Drive/team7/first_try.h5')
my_model.fit_generator(
        train_generator,
        steps_per_epoch=1280,
        epochs=5,
        validation_data=test_generator,
        validation_steps=1280,
        verbose = 2,
        # use_multiprocessing=False
)

Epoch 1/5
1280/1280 - 3231s - loss: 0.3327 - mse: 0.3345 - val_loss: 0.2158 - val_mse: 0.2052
Epoch 2/5
1280/1280 - 3199s - loss: 0.3381 - mse: 0.3375 - val_loss: 0.1905 - val_mse: 0.2052
Epoch 3/5
1280/1280 - 3186s - loss: 0.3415 - mse: 0.3376 - val_loss: 0.1946 - val_mse: 0.2052
Epoch 4/5
1280/1280 - 3270s - loss: 0.3327 - mse: 0.3345 - val_loss: 0.2053 - val_mse: 0.2052
Epoch 5/5
1280/1280 - 3352s - loss: 0.3346 - mse: 0.3377 - val_loss: 0.2029 - val_mse: 0.2052


In [0]:
my_model.save("/content/drive/My Drive/model_ver2.h5");

In [0]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/My Drive/model_ver2.h5')

100/100 [==============================] - 118s 1s/step - loss: 0.1947 - mean_squared_error: 0.2054


[0.19472430646419525, 0.205379918217659]

Train model pruned model

In [0]:
! pip install tensorflow-model-optimization

     |████████████████████████████████| 174kB 4.6MB/s 
     |████████████████████████████████| 296kB 15.1MB/s 


In [0]:
from tensorflow_model_optimization.sparsity import keras as sparsity
import tensorflow as tf
import numpy as np

epochs = 7

In [0]:
puning_params = {
    'pruning_schedule' : sparsity.PolynomialDecay(
        initial_sparsity = 0.5,
        final_sparsity = 0.9,
        begin_step = 5,
        end_step = 90,
        frequency = 5
    )
}


In [0]:
l = tf.keras.layers

model_vgg16_conv = VGG16(weights='imagenet', include_top=False) # data_format = "channels_first"
for layer in model_vgg16_conv.layers[:-4]:
    layer.trainable = False
for layer in model_vgg16_conv.layers:
    print(layer, layer.trainable)

input = Input(shape=(150,150,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten(name='flatten')(output_vgg16_conv)
x = sparsity.prune_low_magnitude(l.Dense(4096, activation='relu'),)(x)
x = sparsity.prune_low_magnitude(l.Dense(1024, activation='relu'),)(x)
x = sparsity.prune_low_magnitude(l.Dense(1, activation='relu'),)(x)
x = l.Dropout(0.1)(x)

pruned_model = Model(input, x)
pruned_model.summary()

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f0bf8376b38> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0bf8376cc0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0bf8376f60> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0bf837d828> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0bf811b128> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0bf8120470> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0bf8128630> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0bf81284a8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0bf812c7f0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0bf80c29b0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0bf80c98d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [0]:
import tempfile
logdir = tempfile.mkdtemp()
print('Writing training logs to ' + logdir)

Writing training logs to /tmp/tmp6cawgvjt


In [0]:
pruned_model.compile(
    loss='mse',
    optimizer='adam',
    metrics=['mse'])

In [0]:
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

In [0]:
pruned_model.fit(
        train_generator,
        steps_per_epoch=1280,
        epochs=5,
        callbacks=callbacks,
        validation_data=test_generator,
        validation_steps=1280,
        verbose = 2,
        )
pruned_model.save("/content/drive/My Drive/model_pruned.h5",include_optimizer=False);

Epoch 1/3
10/10 - 335s - loss: 2.6519 - mse: 2.6519
Epoch 2/3
10/10 - 68s - loss: 0.1766 - mse: 0.1805
Epoch 3/3
10/10 - 11s - loss: 0.8720 - mse: 0.8953


In [0]:
from tensorflow.keras.models import load_model
import tensorflow_hub as hub
p_model = load_model('/content/drive/My Drive/model_pruned.h5',)
p_model.evaluate(test_generator, steps=4)

ValueError: ignored